In [9]:
import ccxt
import os
from dotenv import load_dotenv
import time
import schedule
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo

In [10]:
load_dotenv()

# Retrieve API keys from environment variables
xP_KEY = os.getenv('xP_KEY')
xP_SECRAT = os.getenv('xP_SECRET')

kC_KEY = os.getenv('kC_KEY')
kC_SECRET = os.getenv('kC_SECRAT')
kC_PASSWORD = os.getenv('kC_PASSWORD')




In [11]:
# Initialize the KuCoin exchange object
kucoin = ccxt.kucoin({
    'enableRateLimit': True,
    'apiKey': kC_KEY,
    'secret': kC_SECRET,
    'password': kC_PASSWORD
})
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': xP_KEY,
    'secret': xP_SECRAT
})
symbol = 'BTCUSD'
symbol_str = "STRKUSDTM"

# Helping Function

In [12]:

import json

from kucoin_futures.client import Trade

def get_size_pos(sy):        
    client = Trade(key=kC_KEY, secret=kC_SECRET, passphrase=kC_PASSWORD)
    positions = client.get_position_details(symbol=sy)
    if positions['currentQty'] and positions['isOpen']:
        return positions['currentQty'],positions['isOpen']
    return 0,None

def ask_bid():
    order_book = phemex.fetch_order_book(symbol=symbol)
    return order_book['asks'][0][0], order_book['bids'][0][0]
get_size_pos(sy=symbol_str)


(2, True)

# Daily SMA 20 Days

In [13]:
 #! FIND DAILY SMA PER 20 DAYS

def sma_20d():
    print("starting 20 Days ...")

    timeframe = '1d'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # print(" Got the Bars: ",bars)
    data_20days = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_20days['date'] = pd.to_datetime(data_20days['timestep'], unit='ms')
    data_20days.drop(columns=['timestep'], inplace=True)
    data_20days['SMA_20'] = data_20days['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
    data_20days.loc[data_20days['SMA_20'] > bid, 'SIGNAL'] = 'SELL'
    data_20days.loc[data_20days['SMA_20'] < bid, 'SIGNAL'] = 'BUY'

    if data_20days['SIGNAL'].iloc[-1] == 'SELL':
        print("Opening as sell ...")
    else:
        print('Opening as Buy ...')
    
    return data_20days

sma_20d()


starting 20 Days ...


KeyboardInterrupt: 

#  15 Minutes SMA

In [6]:
 #! Define the SMA for the 15 Minutes With Prices bp_1, bp_2, sp_1, sp_2

def sma_15min():
    print("starting sma 15 Minuts  ...")

    timeframe = '15m'
    num_bars = 100
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)
    # # print(" Got the Bars: ",bars)
    data_15m = pd.DataFrame(bars, columns=['timestep', 'open', 'high', 'low', 'close', 'volume'])
    data_15m['date'] = pd.to_datetime(data_15m['timestep'], unit='ms')
    data_15m.drop(columns=['timestep'], inplace=True)
    data_15m['SMA_15m'] = data_15m['close'].rolling(window=20).mean()

    bid = ask_bid()[1] 
        
    data_15m['bp_1'] = data_15m['SMA_15m'] * 1.001
    data_15m['bp_2'] = data_15m['SMA_15m'] * 0.997
    data_15m['sp_1'] = data_15m['SMA_15m'] * 0.999
    data_15m['sp_2'] = data_15m['SMA_15m'] * 1.003

    return data_15m

sma_15min()

starting sma 15 Minuts  ...


,open,high,low,close,volume,date,SMA_15m,bp_1,bp_2,sp_1,sp_2
0,60944.1,61194.7,60919.6,61118.3,1953809.0,2024-10-02 00:30:00,NaN,NaN,NaN,NaN,NaN
1,61118.2,61118.2,60990.0,61041.0,937185.0,2024-10-02 00:45:00,NaN,NaN,NaN,NaN,NaN
2,61051.8,61278.0,61000.1,61278.0,1049167.0,2024-10-02 01:00:00,NaN,NaN,NaN,NaN,NaN
3,61277.9,61407.0,61184.1,61390.9,681130.0,2024-10-02 01:15:00,NaN,NaN,NaN,NaN,NaN
4,61390.9,61442.4,61302.8,61369.3,298210.0,2024-10-02 01:30:00,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,60803.9,60807.7,60529.6,60735.6,677255.0,2024-10-03 00:15:00,60627.335,60687.962335,60445.452995,60566.707665,60809.217005
96,60751.2,61080.4,60723.5,60848.5,2238136.0,2024-10-03 00:30:00,60665.490,60726.155490,60483.493530,60604.824510,60847.486470
97,60848.6,61068.2,60825.5,60991.3,3255209.0,2024-10-03 00:45:00,60706.060,60766.766060,60523.941820,60645.353940,60888.178180
98,60991.3,61125.3,60973.5,61108.5,1127420.0,2024-10-03 01:00:00,60747.155,60807.902155,60564.913535,60686.407845,60929.396465


# Kill Switch & Pnl Close

In [ ]:
target = 25

def kill_switch():
    print("Starting the Kill switching ...")
    # TODO: Implement the Kill Switch
    pnl_c, _ ,side = pnl_close_kucoin() #! pnl_close, open_pos, Side-long,short
    size_for_kill,open_p, = get_size_pos(sy=symbol_str)

    print(f"open position: {open_p} side: {side} Size postion: {size_for_kill}")
    
    ask,bid = ask_bid()
    while open_p:
        print(f"Killing the size {size_for_kill}")

        if side == 'long':
            print(f" -- -- Make the order for Sell at {ask} With quntety of {size_for_kill} of {symbol_str}")
            print("=== Sleeping 30 Sec =====")
            time.sleep(30)
            # TODO: Existing the SELL the Order. At Ask.

        elif side == 'short':
            print(f" -- -- Make the order for BUY at {bid} With quntety of {size_for_kill} of {symbol_str}")
            print("=== Sleeping 30 Sec =====")
            # TODO: Existing the BUY Order. At bid.
        
        else:
            print(f" -- Position is not Sure, Not kill switching ...")
        _, open_p = get_size_pos()


def pnl_close_kucoin():
    print("Starting PNL ...")
    
    # Example of different parameters

    poc_dic = kucoin.fetch_ticker(symbol='STRK/USDT')
    perc = poc_dic['percentage']
    diff = float(poc_dic['info']['changePrice'])
    current_price = poc_dic['last']
    entry_price = float(poc_dic['open']) 
    side = 'long' if (diff >= 0 and current_price >= entry_price) else 'short'
    if diff != 0:
        side = 'long' if diff > 0 and current_price > entry_price else 'short'
    else:
        side = 'none'

    print(f"Difference: {diff}, Current Price: {current_price}, Entry Price: {entry_price}, Side: {side}, Percentage: {perc}%")


    pnlclose = False
    in_pos = False
    if perc > 0:
        # * We are in a winning position (profit)
        if perc > target:
            # ! Closing the position as target is hit
            print("In Profit also Hit the Target ...")
            pnlclose = True
            kill_switch()
        else:
            print("In Profit but  not hit the target, so wait !!")
    else:
        in_pos = True
        print("We are not in Profit yet.")
    print("Finishing the PNL close ...")

    return pnlclose, in_pos,side

# print(pnl_close_phemex())
# pnl_close_kucoin()
kill_switch()

In [ ]:
def bot():
    data_15m = sma_15min()
    data_20d = sma_20d()
    ask,bid = ask_bid()

    # * Define OPEN Order
    signal = data_20d.iloc[-1]['SIGNAL']
    pnl_close_kucoin()
    
    open_size = 1
    
    in_pos = get_size_pos()[1]


    if not in_pos:
        if signal == "BUY":
            print("Make order as BUY !!")
            bp_1 = data_15m.iloc[-1]['bp_1']
            bp_2 = data_15m.iloc[-1]['bp_2']
            print(f"bp_1: {bp_1} bp_2: {bp_2}")
            # TODO: Make Buy Order Here ...
        else:
            print("Make order as SELL !!")
            sp_1 = data_15m.iloc[-1]['sp_1']
            sp_2 = data_15m.iloc[-1]['sp_2']
            print(f"sp_1: {sp_1} sp_2: {sp_2}")
            # TODO: Make Sell Order Here ... 
    else:
        print("Already In Position, not making new Position")
bot()